In [43]:
### Library Imports
import pandas as pd
import yfinance as yf
import numpy as np
from scipy import stats
import datetime
import matplotlib.pyplot as plt


In [44]:
### Function to Import Stock Data
def import_stock_data(tickers, start_date, end_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date, end_date)['Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date, end_date)['Close']
    
    # Reset index to include the Date as a column
    data = data.reset_index()

    return data

tickers = ['UBER', 'AOS']
start_date = '2025-01-01'
end_date = '2025-06-01'
stock_data = import_stock_data(tickers, start_date, end_date)
print(stock_data.head())


/var/folders/n5/bknqp6qs37l5l4bwv9wv3bs40000gn/T/ipykernel_60806/85346715.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data[tickers] = yf.download(tickers, start_date, end_date)['Close']
[*********************100%***********************]  2 of 2 completed

        Date       UBER        AOS
0 2025-01-02  66.650047  63.169998
1 2025-01-03  67.907227  64.589996
2 2025-01-06  68.204193  66.309998
3 2025-01-07  67.610252  66.150002
4 2025-01-08  67.986420  64.910004


In [45]:
### Compute the Percentage Change
def pct_change(tickers, df):
    # Add col with daily pct change for each ticker
    for t in tickers:
        df['PCT ' + t] = df[t].pct_change()
    
    # Drop null vals
    df.dropna(subset=['PCT ' + t for t in tickers], inplace=True)

    return df

### Function Call to Compute the Percentage Change
stock_data = pct_change(tickers, stock_data)
print(stock_data.head())


        Date       UBER        AOS  PCT UBER   PCT AOS
1 2025-01-03  67.907227  64.589996  0.018862  0.022479
2 2025-01-06  68.204193  66.309998  0.004373  0.026630
3 2025-01-07  67.610252  66.150002 -0.008708 -0.002413
4 2025-01-08  67.986420  64.910004  0.005564 -0.018745
5 2025-01-10  66.897530  65.970001 -0.016016  0.016330


In [46]:
### Generate Buy Signals
''' 
Generate a "BUY" signal if the stock loses more than 5% of its value in a single day
'''
def generate_signals(df, drop_threshold):
    """
    Generate 'BUY' signals if a stock drops more than the threshold in a day.
    """
    for t in tickers:
        df['Signal ' + t] = (df['PCT ' + t] < drop_threshold).astype(int)
        # Test to see the number of generated signals
        print(f"{'Signal ' + t}: {df['Signal ' + t].sum()} buy signals generated")

    return df

### Function Retun to Generate Buy Signals
stock_data = generate_signals(stock_data, drop_threshold = -0.05)
print(stock_data.head())


Signal UBER: 1 buy signals generated
Signal AOS: 3 buy signals generated
        Date       UBER        AOS  PCT UBER   PCT AOS  Signal UBER  \
1 2025-01-03  67.907227  64.589996  0.018862  0.022479            0   
2 2025-01-06  68.204193  66.309998  0.004373  0.026630            0   
3 2025-01-07  67.610252  66.150002 -0.008708 -0.002413            0   
4 2025-01-08  67.986420  64.910004  0.005564 -0.018745            0   
5 2025-01-10  66.897530  65.970001 -0.016016  0.016330            0   

   Signal AOS  
1           0  
2           0  
3           0  
4           0  
5           0  


In [47]:
### Trade Simulation Function
''' 
Enter trade the day after the signal and exit after a fixed holding period (ex. 5 days)
'''
def trade_sim(ticker, df, holding_days):
    ### Empty List to Store Trades
    trades = []

    # Iterate over the df, stopping with the number of holding days left (avoids out of bounds error) 
    for i in range(len(df) - holding_days):
        # Identify the Buy Signals - Locate rows with a "1" indicator
        if df.iloc[i]['Signal ' + ticker] == 1:
            # Set Entry/Exit Date - For each signal date (i), set the entry date to be the next trading day (i + 1)
            entry_idx = i + 1
            exit_idx = i + holding_days

            entry_date = df.index[entry_idx]
            exit_date = df.index[exit_idx]

            # Record Entry and Exit Prices
            entry_price = df.iloc[entry_idx][ticker]
            exit_price = df.iloc[exit_idx][ticker]

            # Compute the Return for the Asset
            returns = (exit_price / entry_price) - 1

            # Save trade to list
            trades.append({
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Entry Price': entry_price,
                'Exit Price': exit_price,
                'Return': returns
            })

    return pd.DataFrame(trades, columns = ['Entry Date', 'Exit Date', 'Entry Price', 'Exit Price', 'Return'])

### Function to Return the Trade Simulation Function
#for t in tickers:
trades = trade_sim('AOS', stock_data, holding_days = 14)
print(trades)


   Entry Date  Exit Date  Entry Price  Exit Price    Return
0          23         36    69.989998   75.870003  0.084012
1          63         76    64.620003   78.099998  0.208604
2          64         77    65.639999   77.750000  0.184491


In [48]:
### Evaluate Total Performance
def performance_metrics(df, holding_days):
    # Total Return
    total_rets = (1 + df['Return']).prod() - 1
    # Avg Return
    avg_rets = df['Return'].mean()
    # Win Rate = Num Winning Trades / Total Trades
    win_rate = (df['Return'] > 0).mean()
    # Sharpe Ratio
    sharpe = (avg_rets / df['Return'].std()) * np.sqrt(252 / holding_days)

    print(f"Number of Trades: {len(df)}")
    print(f"Total Return: {total_rets:.2%}")
    print(f"Average Trade Return: {avg_rets:.2%}")
    print(f"Win Rate: {win_rate:.2%}")
    print(f"Sharpe Ratio: {sharpe:.2f}")

### Function Return to Evaluate Total Performance
metrics = performance_metrics(trades, holding_days = 14)


Number of Trades: 3
Total Return: 55.19%
Average Trade Return: 15.90%
Win Rate: 100.00%
Sharpe Ratio: 10.21
